# Projet NoSQL - GDELP
*Version* - 1.1

*Last Update* - 28.01.2022

In [1]:
import pandas as pd
import glob
import numpy as np

# Global Variables

In [2]:
col_events_name = [
    # --- Dates
    'GlobalEventID', 'Day', 'MonthYear', 'Year', 'FractionDate',
    # --- Actors
    'Actor1Code', 'Actor1Name', 'Actor1CountryCode', 'Actor1KnownGroupCode',
    'Actor1EthnicCode', 'Actor1Religion1Code', 'Actor1Religion2Code',
    'Actor1Type1Code', 'Actor1Type2Code', 'Actor1Type3Code',
    'Actor2Code', 'Actor2Name', 'Actor2CountryCode', 'Actor2KnownGroupCode',
    'Actor2EthnicCode', 'Actor2Religion1Code', 'Actor2Religion2Code',
    'Actor2Type1Code', 'Actor2Type2Code', 'Actor2Type3Code',
    # --- Events
    'IsRootEvent', 'EventCode', 'EventBaseCode', 'EventRootCode', 'QuadClass',
    'GoldsteinScale', 'NumMentions', 'NumSources', 'NumArticles', 'AvgTone',
    # --- Geo
    'Actor1Geo_Type', 'Actor1Geo_Fullname', 'Actor1Geo_CountryCode',
    'Actor1Geo_ADM1Code', 'Actor1Geo_ADM2Code', 'Actor1Geo_Lat',
    'Actor1Geo_Long', 'Actor1Geo_FeatureID',
    'Actor2Geo_Type', 'Actor2Geo_Fullname', 'Actor2Geo_CountryCode',
    'Actor2Geo_ADM1Code', 'Actor2Geo_ADM2Code', 'Actor2Geo_Lat',
    'Actor2Geo_Long', 'Actor2Geo_FeatureID',
    'ActionGeo_Type', 'ActionGeo_Fullname', 'ActionGeo_CountryCode',
    'ActionGeo_ADM1Code', 'ActionGeo_ADM2Code', 'ActionGeo_Lat',
    'ActionGeo_Long', 'ActionGeo_FeatureID',
    # --- General
    'DATEADDED', 'SOURCEURL']

col_mentions_name = ['GlobalEventID', 'EventTimeDate', 'MentionTimeDate',
    'MentionType', 'MentionSourceName', 'MentionIdentifier', 'SentenceID',
    'Actor1CharOffset', 'Actor2CharOffset', 'ActionCharOffset',
    'InRawText', 'Confidence', 'MentionDocLen', 'MentionDocTone',
    'MentionDocTranslationInfo', 'Extras']

col_gkg_name = [''] # --- To be completed

# Functions

In [3]:
def generate_zip_files(initial_date, final_date):
    '''Generates all zip files to download between a user define period of
    time.

    Parameters
    ---------
    - initial_date: type: str, format: YYYYMMDDHHMMSS
    - final_date: type: str, format: YYYYMMDDHHMMSS

    Return
    -----
    - df: type: DataFrame
        All the zip files for English and Translingual : export, mentions
        and gkg
    '''
    # --- Local variables definiton
    url = 'http://data.gdeltproject.org/gdeltv2/'
    file_type = ['.export.CSV.zip', '.mentions.CSV.zip', '.gkg.csv.zip']

    datetime_index = pd.date_range(start=initial_date, end=final_date, freq='15min')

    df = pd.DataFrame([['', '']], columns=['date-str', 'zip'], index=datetime_index)

    df['date-str'] = df.index.strftime('%Y%m%d%H%M%S')
    df['zip'] = df['date-str'].apply(lambda x: url+x)

    for csv in file_type:
        df.insert(loc=df.shape[1],
            column='eng-'+csv.split('.')[1],
            value=df['zip'].apply(lambda x: x+csv))
    
        df.insert(loc=df.shape[1],
            column='translingual-'+csv.split('.')[1],
            value=df['zip'].apply(lambda x: x+'.translation'+csv))

    del df['date-str']
    del df['zip']

    return df

In [4]:
path = '/Users/alann/PythonProjects/INF728-NoSQL/'

def delete_zip_files_used(path):
    list_zip_files = glob.glob(path + '*.zip')
    # --- To be continued
    return list_zip_files

# Main

## Question 1
Afficher le nombre d’articles/évènements qu’il y a eu pour chaque triplet (jour, pays de l’évènement, langue de l’article).

Pour cette question, il est nécessaire d'avoir :
- le jour où un article est écrit : **Day**, table **EVENT**
    - ce qui implique aussi l'id : **GlobalEventID**, table **EVENT**
- le pays de l'évènement : **ActionGeo_CountryCode**, table **EVENT**
- la lanque dans laquelle l'article est écrit : **MentionDocTranslationInfo**, table **MENTIONS**
    - NAN représente l'écriture en Anglais
- tous les articles ayant été écrit ce jour la : **GlobalEventID**, table **EVENT**

*Question* : Pourquoi les tables EVENTS et MENTIONS ne sont pas de la même taille ?

In [5]:
initial_date = '20210101000000'
final_date = '20210101001500'

zip_files = generate_zip_files(initial_date, final_date)
dict_files = dict(zip_files.iloc[0])
dict_files

{'eng-export': 'http://data.gdeltproject.org/gdeltv2/20210101000000.export.CSV.zip',
 'translingual-export': 'http://data.gdeltproject.org/gdeltv2/20210101000000.translation.export.CSV.zip',
 'eng-mentions': 'http://data.gdeltproject.org/gdeltv2/20210101000000.mentions.CSV.zip',
 'translingual-mentions': 'http://data.gdeltproject.org/gdeltv2/20210101000000.translation.mentions.CSV.zip',
 'eng-gkg': 'http://data.gdeltproject.org/gdeltv2/20210101000000.gkg.csv.zip',
 'translingual-gkg': 'http://data.gdeltproject.org/gdeltv2/20210101000000.translation.gkg.csv.zip'}

In [6]:
# --- Eng articles
df_export = pd.read_csv(dict_files['eng-export'].split('/')[-1], 
    sep='\t',
    names=col_events_name,
    header=None)

df_mentions = pd.read_csv(dict_files['eng-mentions'].split('/')[-1], 
    sep='\t',
    names=col_mentions_name,
    header=None)

col_req1_export = ['GlobalEventID', 'Day', 'ActionGeo_CountryCode']
col_req1_mentions = ['GlobalEventID', 'MentionDocTranslationInfo']

df_req1 = df_export[col_req1_export]
df_req1 = df_req1.join(df_mentions[col_req1_mentions], rsuffix='_')
del df_req1['GlobalEventID_']

df_req1

,GlobalEventID,Day,ActionGeo_CountryCode,MentionDocTranslationInfo
0,962219499,20200102,US,NaN
1,962219500,20201202,AS,NaN
2,962219501,20201202,MV,NaN
3,962219502,20201202,RS,NaN
4,962219503,20201225,US,NaN
...,...,...,...,...
2457,962221956,20210101,NaN,NaN
2458,962221957,20210101,UK,NaN
2459,962221958,20210101,UK,NaN
2460,962221959,20210101,UK,NaN


In [7]:
# --- Other countries articles
df_export_translingual = pd.read_csv(dict_files['translingual-export'].split('/')[-1], 
    sep='\t',
    names=col_events_name,
    header=None)

df_mentions_translingual = pd.read_csv(dict_files['translingual-mentions'].split('/')[-1], 
    sep='\t',
    names=col_mentions_name,
    header=None)

col_req1_export_translingual = ['GlobalEventID', 'Day', 'ActionGeo_CountryCode']
col_req1_mentions_translingual = ['GlobalEventID', 'MentionDocTranslationInfo']

df_req1_translingual = df_export_translingual[col_req1_export_translingual]
df_req1_translingual = df_req1_translingual.join(
    df_mentions_translingual[col_req1_mentions_translingual], 
    rsuffix='_')

del df_req1_translingual['GlobalEventID_']

df_req1_translingual['MentionDocTranslationInfo'] = df_req1_translingual[
    'MentionDocTranslationInfo'].apply(
        lambda x: x.split(';')[0].split(':')[-1])

df_req1_translingual

,GlobalEventID,Day,ActionGeo_CountryCode,MentionDocTranslationInfo
0,962223494,20200102,CH,kor
1,962223495,20200102,CH,kor
2,962223496,20200102,CH,kor
3,962223497,20200102,KS,kor
4,962223498,20200102,EI,rus
...,...,...,...,...
551,962224045,20210101,CM,spa
552,962224046,20210101,US,spa
553,962224047,20210101,LY,spa
554,962224048,20210101,UK,spa


In [8]:
df_req1_final = pd.concat([df_req1, df_req1_translingual]).reset_index(drop=True)
df_req1_final

,GlobalEventID,Day,ActionGeo_CountryCode,MentionDocTranslationInfo
0,962219499,20200102,US,NaN
1,962219500,20201202,AS,NaN
2,962219501,20201202,MV,NaN
3,962219502,20201202,RS,NaN
4,962219503,20201225,US,NaN
...,...,...,...,...
3013,962224045,20210101,CM,spa
3014,962224046,20210101,US,spa
3015,962224047,20210101,LY,spa
3016,962224048,20210101,UK,spa


In [9]:
# --- Groupby
df_req1_final.groupby(
    by=['Day', 'ActionGeo_CountryCode', 'MentionDocTranslationInfo']
).count().sort_values(by='GlobalEventID', ascending=False)

GlobalEventID
Day      ActionGeo_CountryCode MentionDocTranslationInfo               
20210101 US                    spa                                   21
         RS                    spa                                   17
         UK                    spa                                   14
         MX                    spa                                   14
         US                    rus                                   11
...                                                                 ...
         KS                    por                                    1
                               ron                                    1
         CM                    por                                    1
         KS                    tel                                    1
         VE                    fra                                    1

[246 rows x 1 columns]

In [10]:
# Test difference between NAN and 'eng' in MentionDocTranslationInfo
df_copy = df_req1_final.copy()
df_copy['MentionDocTranslationInfo'] = df_copy['MentionDocTranslationInfo'].fillna(value='eng')

# --- Groupby test
df_copy.groupby(
    by=['Day', 'ActionGeo_CountryCode', 'MentionDocTranslationInfo']
).count().sort_values(by='GlobalEventID', ascending=False).head(20)

GlobalEventID
Day      ActionGeo_CountryCode MentionDocTranslationInfo               
20210101 US                    eng                                 1242
         UK                    eng                                  236
         CA                    eng                                  102
         AS                    eng                                   70
         FR                    eng                                   55
         CH                    eng                                   49
         NI                    eng                                   48
         IR                    eng                                   43
         IN                    eng                                   35
         IS                    eng                                   30
         BF                    eng                                   26
         IZ                    eng                                   26
         EI                    eng                                   23
         US                    spa                                   21
         JO                    eng                                   21
         RS                    eng                                   20
         JA                    eng                                   19
         SP                    eng                                   18
         RS                    spa                                   17
         EG                    eng                                   16

## Question 2
Pour un pays donné en paramètre, affichez les évènements qui y ont eu place triées par le nombre de mentions (tri décroissant); permettez une agrégation par jour/mois/année

Pour cette question, il est nécessaire d'avoir :
- Les évènements : **GlobalEventID**, table *EVENT*
- Le pays où a lieu des évènements : **ActionGeo_CountryCode**, table *EVENT*
- Le nombre de mensions : **NumMentions**, table *EVENT*

Pour l'agrégation, il faut également :
- Le jour : **Day**, table *EVENT*
- Le mois : **MonthYear**, table *EVENT*
- L'année : **Year**, table *EVENT*

In [11]:
# --- Pour rappel : liste de tous les pays visibles.
df_export['ActionGeo_CountryCode'].unique()

array(['US', 'AS', 'MV', 'RS', nan, 'FR', 'IR', 'SF', 'IN', 'UK', 'BG',
       'BF', 'BR', 'CA', 'NI', 'BE', 'CH', 'HK', 'IS', 'GM', 'NP', 'EG',
       'SP', 'FJ', 'EI', 'LH', 'NO', 'TW', 'IC', 'IZ', 'JO', 'NZ', 'IT',
       'MX', 'SO', 'SA', 'YM', 'JA', 'VM', 'AF', 'AE', 'BU', 'GJ', 'BB',
       'MY', 'SZ', 'KS', 'TU', 'CM', 'CD', 'GZ', 'BH', 'HR', 'DA', 'LS',
       'RO', 'GI', 'PC', 'GH', 'BK', 'ET', 'KN', 'SU', 'WE', 'PP', 'GR',
       'LE', 'GB', 'GG', 'UP', 'RQ', 'JE', 'RP', 'QA', 'TH', 'TS', 'VT',
       'ZI', 'ML'], dtype=object)

In [12]:
# --- Eng articles
col_req2_export = ['GlobalEventID', 'Day', 'MonthYear', 'Year',
    'ActionGeo_CountryCode', 'NumMentions']

df_req2 = df_export[col_req2_export]

# --- Other countries articles
df_req2_translingual = df_export_translingual[col_req2_export]

# --- Concat
df_req2_final = pd.concat([df_req2, df_req2_translingual]).reset_index(drop=True)
df_req2_final

,GlobalEventID,Day,MonthYear,Year,ActionGeo_CountryCode,NumMentions
0,962219499,20200102,202001,2020,US,4
1,962219500,20201202,202012,2020,AS,4
2,962219501,20201202,202012,2020,MV,5
3,962219502,20201202,202012,2020,RS,8
4,962219503,20201225,202012,2020,US,4
...,...,...,...,...,...,...
3013,962224045,20210101,202101,2021,CM,10
3014,962224046,20210101,202101,2021,US,4
3015,962224047,20210101,202101,2021,LY,6
3016,962224048,20210101,202101,2021,UK,2


In [13]:
choice = np.random.choice(df_export['ActionGeo_CountryCode'].unique())
print('The random choice is: ', choice)

df_choice = df_req2_final[df_req2_final['ActionGeo_CountryCode'] == choice]
df_choice

The random choice is:  GI


,GlobalEventID,Day,MonthYear,Year,ActionGeo_CountryCode,NumMentions
990,962220489,20210101,202101,2021,GI,5
993,962220492,20210101,202101,2021,GI,1
994,962220493,20210101,202101,2021,GI,4
2698,962223730,20210101,202101,2021,GI,10


In [14]:
# --- Answer the question 
agg = 'Day'  # Choice between: Day, MonthYear, Year
df_choice.groupby(by=['ActionGeo_CountryCode', agg]).agg({'NumMentions':'sum'})

,,NumMentions
ActionGeo_CountryCode,Day,
GI,20210101,20
